In [ ]:
# Code pour créer des nouvelles colonnes

In [ ]:
# Montant réellement payé / Montant prescrit
installments_payments['PAYMENT_PERC'] = installments_payments['AMT_PAYMENT'] / installments_payments['AMT_INSTALMENT']

# Montant prescrit - Montant réellement payé
installments_payments['PAYMENT_DIFF'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']

# Différence brute des jours de paiement
diff_days = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']

# DPD (Days Past Due): Retard uniquement (diff_days > 0, sinon 0)
installments_payments['DPD'] = diff_days.clip(lower = 0)

# DBD (Days Before Due): Avance uniquement (diff_days < 0, sinon 0)
installments_payments['DBD'] = (-diff_days).clip(lower = 0)

# Flag pour les paiements en retard (DPD > 0)
installments_payments['DPD_FLAG'] = (installments_payments['DPD'] > 0).astype(int)

In [ ]:
#Code pour réduire la taille des données

In [ ]:
def reduce_mem_usage(df):
    """
    Itère sur toutes les colonnes d'un DataFrame et réduit la précision
    des types numériques (int et float) pour diminuer la consommation de mémoire.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Usage mémoire initial du DataFrame: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        # Traiter uniquement les colonnes numériques
        if col_type != object and col_type != str and col_type != bool:
            c_min = df[col].min()
            c_max = df[col].max()

            # --- Conversion des entiers (Integers) ---
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # --- Conversion des décimaux (Floats) ---
            else:
                # La majorité de vos colonnes d'agrégats (mean, var, proportions) sont ici
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    # Conversion principale : float64 -> float32
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64) # Garder float64 si la précision est nécessaire

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Usage mémoire final du DataFrame: {end_mem:.2f} MB")
    print(f"Mémoire réduite de {(start_mem - end_mem) / start_mem * 100:.1f} %")

    return df